---
title: Week 2 Orchestra Simulation
subtitle: Orchestra Scenario, Fluorite & Gypsum simulation
author:
  - name: Timo Heimovaara
    affiliations: Delft University of Technology, department of Geoscience & Engineering
    orcid: 
    email: t.j.heimovaara@tudelft.nl
license: CC-BY-NC-ND-4.0 (https://creativecommons.org/licenses/by-nc-nd/4.0/).
date: 2025-12-10
kernelspec:
    name: python3
    display_name: 'Python 3.13'
---

# Week 2 Full simulation with ORCHESTRA Fluorite-Gypsum
## Orchestra as a tool
We will work using a limited set of chemical systems to illustrate the chemical equilibrium concepts the students need to understand.

We setup a default system for Orchestra. Using python we will change the total amounts, gas and liquid volumes.
We will also setup different systems for open and closed systems... 


In [1]:
# Import libraries
import os
import sys
from IPython.utils import capture
from IPython.display import display, Markdown

import numpy as np
import matplotlib.pyplot as plt
import PyORCHESTRA # here, the ORCHESTRA submodule is imported
import pandas as pd
import seaborn as sns

%matplotlib widget
sns.set()

# Prepare a file to capture PyOrchestra output
capture_file = open("pyorchestra_output.log", "w")

# pyOrchestra is implemented in C++
# Save original stdout file descriptor
original_stdout_fd = sys.stdout.fileno()

# Duplicate original stdout so we can restore it later
saved_stdout_fd = os.dup(original_stdout_fd)


Explain in a step wise approach the following code section
ToDO rename the Orchestra files, update all files so that they are well documented and describe the system

create a version for a fixed pressure infinite volume, a version for variable pressure fixed volume and fixed volume variable pressure.

pyorchestra will be initialize with the correct chemistry file...

explain that InVars are the species used to define the totals. 
explain that OutVars is the list of Orchestra variables that are exported to python for post-processing or necessary for simulation.

Once prinicple problem is known we can create and initialize a pyorchestra object.

## Define chemical system: Fluorite, Gypsum, water
In order to solve a chemical equilibrium problem with pyOrchestra, we need to define our chemical system first. Orchestra defines this system with a number of text files. The most important one is the so-called chemistry input file. This file is most easily made using the Orchestra GUI which can be accessed by clicking on the orchestrat2023.jar file.

The most convenient approach is to start with a skeleton problem and then adjust the files step by step. 

Using the Orchestra GUI we need to define a system of the following primary entities:

Initial total mass of Ca+2 and F- comes from example. Please note in molar concentration!

|Primary entity|Input Variable|Fix log activity|Log activity|Concentration|Phase|Expression|
|--|--|--|--|--|--|--|
|Ca+2||||2.49e-4|tot||
|F-||||2.89e-4|tot||
|Gypsum[s]||||1.0|tot||
|H|pH|x|7.0|||H.logact=-pH|
|H2O||x|-0.0||||

We allow Orchestra to manage the charge balance with the pH.

Orchestra will use these primary entities, and their initial values to calculate the total elemental composition in the complete system. The fixed log-activties indicate that the total can vary during the simulation, as long as the log-activity condition is maintained. 



## Initialise pyOrchestra
We will use Orchestra to calculate Point C in the example from Appelo & Postma chapter 4.1. We start from the initial situation without any Gypsum.


In [2]:
#--- Initialize problem --- 

# Input file is generated with Orchestra GUI
InputFile = 'chemistry1.inp'
NoCells = 1 #only 1 cell to have a 0-D system 

# We define the input variables that will be changed in the script
InVars = np.array(['Ca+2.tot', 'F-.tot', 
                   'Gypsum[s].tot',
                   ])

# We select the output from Orchestra we need to use
OutVars = np.array(['pH', 'H+.con', 'H+.tot', 
                    'SO4-2.tot', 'SO4-2.con', 'SO4-2.logact',
                    'HSO4-.tot', 'HSO4-.con', 'HSO4-.logact',
                    'Ca+2.tot', 'Ca+2.con', 'Ca+2.logact',
                    'CaF+.con', 'CaOH+.con', 'CaSO4.con',
                    'F-.tot', 'F-.con', 'F-.logact',
                    'Gypsum[s].si', 'Gypsum[s].tot', 'Gypsum[s].un',
                    'Fluorite[s].si', 'Fluorite[s].tot', 'Fluorite[s].un',
                    'I', 'chargebalance'])

p = PyORCHESTRA.ORCHESTRA()
p.initialise(InputFile, NoCells, InVars, OutVars)


Reading and expanding calculator new stylechemistry1.inp
Scanning file: chemistry1.inp
Including file: objects2022.txt
Scanning file: chemistry1.inp
Including file: objects2022.txt
Including file: chemistry1.inp
Including file: objects2022.txt
0.017 sec.
	Reading variables .... 0.007 s
testing:
7:Ca+2.tot
9:F-.tot
11:Gypsum[s].tot
12:pH
18:H+.con
19:H+.tot
20:SO4-2.tot
21:SO4-2.con
22:SO4-2.logact
23:HSO4-.tot
24:HSO4-.con
25:HSO4-.logact
7:Ca+2.tot
26:Ca+2.con
6:Ca+2.logact
27:CaF+.con
28:CaOH+.con
29:CaSO4.con
9:F-.tot
30:F-.con
8:F-.logact
31:Gypsum[s].si
11:Gypsum[s].tot
13:Gypsum[s].un
32:Fluorite[s].si
33:Fluorite[s].tot
15:Fluorite[s].un
3:I
5:chargebalance
Initialise Completed! the following IO parameters will be used:
0 : Node_ID : 0
1 : minTol : 0.001
2 : H2O.logact : 0
3 : I : 0.1
4 : T : 298.15
5 : chargebalance : 0
6 : Ca+2.logact : -15
7 : Ca+2.tot : 0.000249
8 : F-.logact : -15
9 : F-.tot : 0.000289
10 : Gypsum[s].logact : -15
11 : Gypsum[s].tot : 1
12 : pH : -7
13 : Gyp

## Define Initial Conditions (Point A)
Aqueous solution with $\text{Ca}^{+2}$ and $\text{F}^-$, without Gypsum.

In [3]:
# %%
# Initialize values in InVars vector
# InVars need to contain floats
IN = np.array([np.ones_like(InVars)]).astype(float)

IN[0][np.where(InVars == 'Ca+2.tot')] = 2.49e-4 # moles
IN[0][np.where(InVars == 'F-.tot')] = 2.89e-4 # moles
IN[0][np.where(InVars == 'Gypsum[s].tot')] = 1e-20 # No Gypsum (initial conditions)


In [4]:
# Calculate equilibrium conditions for initial situation (Point A in chapter 4.1)
OUT = p.set_and_calculate(IN)
Initial_Res = pd.DataFrame(OUT,columns=OutVars)


## Define final conditions (Point C)
Aqueous solution with $\text{Ca}^{+2}$ and $\text{F}^-$, in equilibrium with Gypsum.

In [5]:
# %%
# Initialize values in InVars vector
# InVars need to contain floats
IN = np.array([np.ones_like(InVars)]).astype(float)

IN[0][np.where(InVars == 'Ca+2.tot')] = 2.49e-4 # moles
IN[0][np.where(InVars == 'F-.tot')] = 2.89e-4 # moles
IN[0][np.where(InVars == 'Gypsum[s].tot')] = 1 # relatively large amount of Gypsum

# %%

In [6]:
OUT = p.set_and_calculate(IN)
Final_Res = pd.DataFrame(OUT,columns=OutVars)

## Process Output

In [12]:
# Process results
deltaCa = (Final_Res['Ca+2.con'] - Initial_Res['Ca+2.con']).values[0]
deltaF = (Final_Res['F-.con'] - Initial_Res['F-.con']).values[0]
deltaSO4 = (Final_Res['SO4-2.con'] - Initial_Res['SO4-2.con']).values[0]

out_df = pd.concat([Initial_Res[['Ca+2.con','F-.con','SO4-2.con', 'Ca+2.logact','F-.logact','SO4-2.logact']],
                    Final_Res[['Ca+2.con','F-.con','SO4-2.con', 'Ca+2.logact','F-.logact','SO4-2.logact']]])
out_df.index = ['initial','final']
out_df.loc['delta',['Ca+2.con', 'F-.con', 'SO4-2.con']] = out_df.loc['final',['Ca+2.con', 'F-.con', 'SO4-2.con']] - out_df.loc['initial',['Ca+2.con', 'F-.con', 'SO4-2.con']]


## Results
See if we can also output the results in a markdown cell.

The change in calcium concentration due to dissolution of Gypsum is: {eval}`format(deltaCa,".3f")` mol/l.

The change in sulphate concentration due to dissolution of Gypsum is: {eval}`format(deltaSO4,".3f")` mol/l.

The change in fluoride concentration due to dissolution of Gypsum is: {eval}`format(deltaF*1000,".3f")` mmol/l.


{eval}`out_df`
